In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from string import punctuation

In [2]:
df = pd.read_pickle('C:/Users/Samarth/Desktop/Mtech AI/NLP/Data/first_pickle.pkl')

In [3]:
df = df[["Consumer complaint narrative","Product"]]

In [4]:
df.dropna(inplace = True)

In [5]:
df = df[:1000]

In [6]:
df.shape

(1000, 2)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer(stop_words='english')

In [9]:
vector = tfidf.fit_transform(df['Consumer complaint narrative'])

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le = LabelEncoder()
y = le.fit_transform(df['Product'])

In [12]:
x = df['Consumer complaint narrative']

In [13]:
from sklearn.decomposition import TruncatedSVD

In [14]:
svd_model = TruncatedSVD(n_components=5,algorithm='randomized',n_iter=100,random_state=42)
svd_model.fit(vector)

TruncatedSVD(algorithm='randomized', n_components=5, n_iter=100,
       random_state=42, tol=0.0)

In [15]:
len(svd_model.components_)

5

In [16]:
terms = tfidf.get_feature_names()
topics = []
for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    topics.append("Topic "+str(i)+": ")
    for t in sorted_terms:
        topics.append(t[0])
final_topic_list = [topics[i:i+8] for i in range(0, len(topics), 8)]
for x in final_topic_list:
    print (x)

['Topic 0: ', 'xxxx', 'xx', 'credit', 'account', '00', 'debt', 'loan']
['Topic 1: ', 'xx', 'loan', 'xxxx', 'mortgage', '00', 'payment', 'payments']
['Topic 2: ', 'xx', 'credit', 'account', 'report', 'debt', 'card', 'bank']
['Topic 3: ', 'xx', 'xxxx', 'report', 'debt', 'collection', 'experian', 'equifax']
['Topic 4: ', 'debt', 'loan', 'company', 'collection', 'agency', 'collector', 'mortgage']


In [17]:
df.head()

,Consumer complaint narrative,Product
1,I have outdated information on my credit repor...,Credit reporting
2,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan
7,An account on my credit report has a mistaken ...,Credit reporting
12,This company refuses to provide me verificatio...,Debt collection
16,This complaint is in regards to Square Two Fin...,Debt collection


In [18]:
first_five = df[:5]

In [19]:
first_five

,Consumer complaint narrative,Product
1,I have outdated information on my credit repor...,Credit reporting
2,I purchased a new car on XXXX XXXX. The car de...,Consumer Loan
7,An account on my credit report has a mistaken ...,Credit reporting
12,This company refuses to provide me verificatio...,Debt collection
16,This complaint is in regards to Square Two Fin...,Debt collection


In [20]:
stop_words = list(set(stopwords.words('english')))+list(punctuation)+['xxx','xxxx','xxxxxxxxxxxx',
  'xxxx/xxxx/xxxx']

In [21]:
lem = WordNetLemmatizer()

In [32]:
text = list(first_five['Consumer complaint narrative'].values)

In [54]:
def clean(text):
    text = text.lower()
    word = word_tokenize(text)
    words = [w for w in word if w not in stop_words and len(w)>3]
    lemm = [lem.lemmatize(w,'v') for w in words]
    return lemm

In [55]:
t = [clean(te) for te in text]

In [57]:
import gensim
from gensim import corpora
#creating the dictionary id2word from our cleaned word list doc_clean
dictionary = corpora.Dictionary(t)
#creating the corpus
doc_term_matrix = [dictionary.doc2bow(doc) for doc in t]

C:\Users\Samarth\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [58]:
print ([[(dictionary[id], freq) for id, freq in cp] for cp in doc_term_matrix[:1]])

[[('credit', 2), ('dispute', 1), ('information', 2), ('meet', 1), ('outdated', 1), ('previously', 1), ('remove', 1), ('report', 2), ('requirements', 1), ('seven', 1), ('years', 1)]]


In [59]:
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(corpus = doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)
print(ldamodel.print_topics(num_topics=5, num_words=7))

[(0, '0.043*"check" + 0.037*"bank" + 0.032*"state" + 0.032*"loan" + 0.032*"citizens" + 0.027*"payment" + 0.022*"receive"'), (1, '0.076*"letter" + 0.035*"experian" + 0.022*"mail" + 0.022*"representative" + 0.022*"dispute" + 0.022*"credit" + 0.022*"report"'), (2, '0.040*"debt" + 0.022*"verification" + 0.022*"company" + 0.022*"provide" + 0.022*"mine" + 0.022*"refuse" + 0.022*"fdcpa"'), (3, '0.036*"square" + 0.036*"financial" + 0.031*"number" + 0.025*"case" + 0.019*"information" + 0.019*"regard" + 0.019*"include"'), (4, '0.005*"years" + 0.005*"meet" + 0.005*"remove" + 0.005*"requirements" + 0.005*"outdated" + 0.005*"seven" + 0.005*"previously"')]


In [64]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\Samarth\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.171706  0.088074       1        1  38.100128
3      0.177873  0.064107       2        1  32.474026
1     -0.022008 -0.164668       3        1  26.353333
2      0.008748  0.004453       4        1   2.825644
4      0.007094  0.008033       5        1   0.246868, topic_info=     Category      Freq            Term     Total  loglift  logprob
term                                                               
47    Default  8.000000          letter  8.000000  30.0000  30.0000
24    Default  6.000000           check  6.000000  29.0000  29.0000
20    Default  5.000000            bank  5.000000  28.0000  28.0000
203   Default  4.000000          square  4.000000  27.0000  27.0000
174   Default  4.000000       financial  4.000000  26.0000  26.0000
21    Default  3.000000         believe  3.000000  25.0000  25.0000
104   Default  3.000000        experian  3.000000  24.0000  24.0000
48    Default  5.000000            loan  5.000000  23.0000  23.0000
26    Default  5.000000        citizens  5.000000  22.0000  22.0000
188   Default  4.000000          number  4.000000  21.0000  21.0000
98    Default  1.000000            debt  1.000000  20.0000  20.0000
61    Default  4.000000         payment  4.000000  19.0000  19.0000
156   Default  3.000000            case  3.000000  18.0000  18.0000
77    Default  5.000000           state  5.000000  17.0000  17.0000
45    Default  3.000000        interest  3.000000  16.0000  16.0000
54    Default  3.000000           money  3.000000  15.0000  15.0000
7     Default  2.000000          report  2.000000  14.0000  14.0000
0     Default  2.000000          credit  2.000000  13.0000  13.0000
1     Default  2.000000         dispute  2.000000  12.0000  12.0000
125   Default  2.000000  representative  2.000000  11.0000  11.0000
143   Default  1.000000           right  1.000000  10.0000  10.0000
157   Default  2.000000            cfpb  2.000000   9.0000   9.0000
160   Default  2.000000       complaint  2.000000   8.0000   8.0000
185   Default  2.000000            list  2.000000   7.0000   7.0000
177   Default  2.000000            form  2.000000   6.0000   6.0000
153   Default  2.000000            cach  2.000000   5.0000   5.0000
2     Default  4.000000     information  4.000000   4.0000   4.0000
68    Default  5.000000         receive  5.000000   3.0000   3.0000
133   Default  1.000000      validation  1.000000   2.0000   2.0000
12    Default  2.000000      additional  2.000000   1.0000   1.0000
...       ...       ...             ...       ...      ...      ...
96     Topic5  0.004448         correct  1.187829   0.4167  -5.3799
126    Topic5  0.004448         request  1.187829   0.4167  -5.3799
120    Topic5  0.004448          offend  1.187829   0.4167  -5.3799
129    Topic5  0.004448           speak  1.187829   0.4167  -5.3799
113    Topic5  0.004448           imply  1.187829   0.4167  -5.3799
123    Topic5  0.004448            real  1.187829   0.4167  -5.3799
112    Topic5  0.004448     implication  1.187829   0.4167  -5.3799
111    Topic5  0.004448          ignore  1.187829   0.4167  -5.3799
119    Topic5  0.004448         mistake  1.187829   0.4167  -5.3799
132    Topic5  0.004448       unhelpful  1.187829   0.4167  -5.3799
117    Topic5  0.004448          lawful  1.187829   0.4167  -5.3799
108    Topic5  0.004448            find  1.187829   0.4167  -5.3799
107    Topic5  0.004448          figure  1.187829   0.4167  -5.3799
2      Topic5  0.004448     information  4.821419  -0.9842  -5.3799
143    Topic5  0.004448           right  1.437017   0.2263  -5.3799
133    Topic5  0.004448      validation  1.390932   0.2589  -5.3799
7      Topic5  0.004448          report  2.585956  -0.3613  -5.3799
0      Topic5  0.004448          credit  2.585956  -0.3613  -5.3799
21     Topic5  0.004448         believe  3.014919  -0.5147  -5.3799
98     Topic5  0.004448            debt  1.